In [ ]:
import numpy as np
from interpol import Interpolation
import matplotlib.cm as cm
import meshplot as mp
import open3d as o3d
from scipy.spatial import KDTree

In [ ]:
mesh = o3d.io.read_triangle_mesh('../../data/humans/mesh/test_15070.off')

skeleton = []
interpolations = []
with open('../../data/humans/centerline/faust_15070.txt') as file:
    for id, line in enumerate(file):
        contenidos = line.split(" ")
        vertices = np.reshape( np.array( contenidos[1:]).astype(np.float32), ((len(contenidos) - 1) // 3, 3) )
        skeleton.append( vertices )
        interpolations.append( Interpolation(vertices) )

vertice_de_arti = {}
with open('../../data/humans/centerline/faust_15070.polylines.txt') as file:
    for line in file:
        s = line.split(' ')
        art = np.array(s[1:4]).astype(np.float32)

        if tuple(art) in vertice_de_arti:
            vertice_de_arti[tuple(art)].append(np.array(s[4:]).astype(np.float32))
        else:
            vertice_de_arti[tuple(art)] = [np.array(s[4:]).astype(np.float32)]

print( len(vertice_de_arti))

In [ ]:
len(np.concatenate(skeleton))

In [ ]:
begs = np.array(list(vertice_de_arti.keys()))

a = np.concatenate( [ np.concatenate(skeleton), np.unique(begs, axis=0)])
print(len(np.unique(a, axis=0)), len(a))

In [ ]:
ends = np.concatenate(list(vertice_de_arti.values()))

print( len(np.unique( np.concatenate([ends, np.asarray(mesh.vertices)]), axis=0 )), len(ends), len(np.asarray(mesh.vertices)) )

# OSEA - Los vertices tienen distinta presicion :/

In [ ]:
len(np.concatenate(list(vertice_de_arti.values())))

In [ ]:
vertices_de_centro = {}
centros = None
for vertices, curva in zip(skeleton, interpolations):
    #print('CURVA:')
    arterias_t = curva.sampleByCurvature(pointsPerUnit=5)
    arterias = list( curva.evaluateFrom(arterias_t) )
    centros = np.concatenate( [centros, arterias] ) if centros is not None else arterias

    l = 0
    for i, art in zip(range(len(arterias_t)), arterias):
        if i == len(arterias_t) - 1:
            h= len(vertices)
        else:
            h = np.ceil( len(vertices) * (arterias_t[i] + (arterias_t[i + 1] - arterias_t[i])/2) ).astype(np.uint32)
        
        #print('   ', l, h)
        if tuple(art) in vertices_de_centro:
            vertices_de_centro[tuple(art)] = np.unique( np.concatenate( [ vertices_de_centro[tuple(art)], np.concatenate( [ vertice_de_arti[tuple(v)] for v in vertices[l:h]] )  ]), axis=0)
        else:
            vertices_de_centro[tuple(art)] = np.concatenate( [ vertice_de_arti[tuple(v)] for v in vertices[l:h]] )
        l = h

In [ ]:
len(np.concatenate(list(vertices_de_centro.values())))

In [ ]:
p = mp.plot( centros, return_plot=True, shading={'point_size': 0.1, 'point_color':'#AA0000' } )

cmap = cm.get_cmap('prism')
for i, (art, verts) in enumerate(vertices_de_centro.items()):

    c = '#'
    for x in (np.array(cmap( (i)/len(centros)) )[0:3] * 255).astype(np.uint32):
        h = hex(x)[2:]
        c += h if len(h) == 2 else '0' + h
    
    p.add_points( np.array( verts ),shading={'point_size': 0.05, 'point_color':c } )

In [ ]:
# unificacion vertices

treeVert = KDTree( np.asarray(mesh.vertices) )

recs = []
for i, (art, verts) in enumerate(vertices_de_centro.items()):
    query = treeVert.query( verts, k=1 )
    newMesh = o3d.geometry.TriangleMesh( mesh  )
    newMesh.remove_vertices_by_mask( [i not in query[1] for i in range(len(np.asarray(mesh.vertices)) + 1)] )
    recs.append( newMesh )

In [ ]:
p3 = mp.plot( np.zeros((1,3)) , return_plot=True)
cmap = cm.get_cmap('flag')

for i, m in enumerate(recs):
    p3.add_mesh( np.asarray(m.vertices), np.asarray(m.triangles), c=np.array(cmap((i)/len(recs)))[0:3])